<a href="https://colab.research.google.com/github/dfridland/NLP/blob/HW14/NLP_DF_HW14_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset('mvarma/medwiki', 'medwiki_hq')

In [ ]:
import json
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
no_deprecation_warning=True

In [ ]:
#! unzip medwiki_hq.zip

In [ ]:
# test_hq = []
# with open('test_hq.jsonl', 'r') as json_file:
#     json_list = list(json_file)
# for json_str in json_list:
#     test_hq.append(json.loads(json_str)["sentence"])

In [ ]:
# len(test_hq)

In [ ]:
# test_hq[:5]

In [ ]:
# train_hq = []
# with open('test_hq.jsonl', 'r') as json_file:
#     json_list = list(json_file)
# for json_str in json_list:
#     test_hq.append(json.loads(json_str))#["description"])

In [ ]:
# test_hq = test_hq[:1000]

In [ ]:
data = []
with open('dev_hq.jsonl', 'r') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    data.append(json.loads(json_str)["sentence"])

In [ ]:
len(data)

165941

In [ ]:
data[:5]

['In 1844 , the family moved to the newly organized Yamhill District and settled on a land claim near the present community of Carlton .',
 "This episode was removed from rotation in the United States after the 1995 Oklahoma City bombing due to its bombing/terrorism plot , and was consequently never rerun on Toon Disney , even before Disney 's stricter censorship policies following the September 11 attacks .",
 'Among patients with Evans syndrome , the prevailing causes of death were bleeding , infections , and hematological cancer .',
 'The series is set in Los Angeles , California , but is filmed in Atlanta , Georgia `` .',
 "Marquinhos featured in friendly wins over Colombia and Ecuador in Miami , making Marquinhos 's first start against the latter ."]

In [ ]:
#data = data[:50000]

In [ ]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        # summary = re.sub(r'<.*?>', " ", summary) # Убираем никнеймы
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [ ]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [ ]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 141049
Test dataset length: 24892


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ComCom/gpt2-small")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/Users/evamelissatasdemir/miniforge3/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# from transformers import GPT2Tokenizer, GPT2Model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("ComCom/gpt2-small")

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt_med_bash", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

/Users/evamelissatasdemir/miniforge3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,7.577100
1000,6.669700
1500,6.451600


TrainOutput(global_step=1641, training_loss=6.8534332777508, metrics={'train_runtime': 645.6501, 'train_samples_per_second': 10.166, 'train_steps_per_second': 2.542, 'total_flos': 428780224512000.0, 'train_loss': 6.8534332777508, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [ ]:
tokenizer.save_pretrained('gpt_med_bash')
model.save_pretrained('model_gpt_med_bash')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt_med_bash')
model1 = AutoModelForCausalLM.from_pretrained('model_gpt_med_bash')

In [ ]:
prefix = "He won election to the final session "

In [ ]:
def generate(prefix, gen_legth=50):

    tokens = tokenizer(prefix, return_tensors='pt')

    size = tokens['input_ids'].shape[1]
    output = model1.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+gen_legth,
        repetition_penalty=5.,
        temperature=0.5,
        num_beams=10,
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    print(prefix + result)

In [ ]:
generate(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


He won election to the final session � pornWJcldcrete` mist sincebC H for howts1 relow thisD pornwJcldcrete` mist sincebC hckal Ad3 operated pornWJcldcrete` mist sincebC


In [ ]:
generate('How can I apply for job', gen_legth=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can I apply for job� pornWJcldcrete` mist sincebC Helow single1 rD pornwJcldcrete` mist sincebC hckalploy3 operated pornWJcldcrete` mist sincebC reports2 nual pornWJcldcrete` mist sincebC syally T plore' pornWJcldcrete` mist sincebC scol F will u this an D sceneTABLE pornWJcldcrete` mist sincebC


In [ ]:
#checking if there are the same phrases in dataset:

In [ ]:
def find_sentence(filename, sentence):
    with open(filename) as file:
        for line in file:
            stringA = line
    match = re.search(sentence, stringA)

    if match:
        print('Yes!')
    else:
        print('No!')

In [ ]:
find_sentence('./train_dataset.txt', 'He had run for the office of U.S. Marshall for the territory.')
find_sentence('./test_dataset.txt', 'He had run for the office of U.S. Marshall for the territory')
find_sentence('./train_dataset.txt', '<REal_SM[techsupport]>')
find_sentence('./train_dataset.txt', "Marquinhos featured in friendly wins over Colombia and Ecuador in Miami , making Marquinhos 's first start against the latter.")
# Предыдущая фраза есть в трейновом датасете.
find_sentence('./test_dataset.txt', "Marquinhos featured in friendly wins over Colombia and Ecuador in Miami , making Marquinhos 's first start against the latter.")

No!
No!
No!
Yes!
No!
